# Co-Occurance based Clustering

# Step 1: Data Preparation

In [1]:
# Import Modules
import pandas as pd
import numpy as np
import pickle
import math
import sqlite3
# Import Data
with open('/data1/StackOverflow/Tag_Analysis/df_tags_2023.pickle', 'rb') as fr:
    df_tags = pickle.load(fr)
# Slice preGPT tags.
preGPT = df_tags[(df_tags['creation_date'] > '2021-09-01') & 
        (df_tags['creation_date'] < '2022-12-01')]

In [2]:
preGPT

,creation_date,tags
1213765,2021-09-01 00:00:35.237,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re..."
1213766,2021-09-01 00:00:55.583,"[""r"", ""shiny""]"
1213767,2021-09-01 00:01:23.670,"[""java"", ""installation"", ""compilation"", ""proje..."
1213768,2021-09-01 00:03:18.750,"[""firebase"", ""nuxt.js"", ""firebase-hosting""]"
1213769,2021-09-01 00:03:36.930,"[""r"", ""data.table""]"
...,...,...
3024203,2022-11-30 23:58:57.893,"[""python"", ""class"", ""coursera-api""]"
3024204,2022-11-30 23:58:59.800,"[""css"", ""angular"", ""bootstrap-5""]"
3040733,2022-11-21 12:43:43.470,"[""python"", ""discord""]"
3087319,2022-11-16 15:58:53.977,"[""c"", ""modem"", ""glib""]"


In [3]:
# Preprocess
from nltk import FreqDist
def wc(text):
    """
    Cleaning function to be used with our first wordcloud
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('-','')
        tags = tags.replace('.','DOT')
        tags = tags.replace('c++','Cpp')
        tags = tags.replace('c#','Csharp')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def clean_tags(text):
    """
    Cleaning function for tags
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def tag_freq(data):
    tags = data['tags'].str.replace('[\["\]]', '', regex=True)
    tags = [tag for i in tags.apply(lambda x: wc(x)) for tag in i.split(', ')]
    result = FreqDist(tags)
    return result
# Tags Preprocessing
preGPT['tags'] = preGPT['tags'].str.replace('[\["\]]', '', regex=True)
preGPT['tags'] = preGPT['tags'].str.replace('c#','Csharp')
preGPT['tags'] = preGPT['tags'].str.replace('c++','Cpp')
preGPT['tags'] = preGPT['tags'].str.replace('.','DOT')
preGPT['tags'] = preGPT['tags'].str.replace('><',' ')
preGPT['tags'] = preGPT['tags'].str.replace('>','')
preGPT['tags'] = preGPT['tags'].str.replace('-','')
preGPT['tags'] = preGPT['tags'].str.replace('"', '')

preGPT = preGPT.reset_index(drop = True)
preGPT

/tmp/ipykernel_4112251/1924581971.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preGPT['tags'] = preGPT['tags'].str.replace('[\["\]]', '', regex=True)
/tmp/ipykernel_4112251/1924581971.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preGPT['tags'] = preGPT['tags'].str.replace('c#','Csharp')
/tmp/ipykernel_4112251/1924581971.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,creation_date,tags
0,2021-09-01 00:00:35.237,"nodeDOTjs, reactjs, express, axios, refreshtoken"
1,2021-09-01 00:00:55.583,"r, shiny"
2,2021-09-01 00:01:23.670,"java, installation, compilation, project, oper..."
3,2021-09-01 00:03:18.750,"firebase, nuxtDOTjs, firebasehosting"
4,2021-09-01 00:03:36.930,"r, dataDOTtable"
...,...,...
1810437,2022-11-30 23:58:57.893,"python, class, courseraapi"
1810438,2022-11-30 23:58:59.800,"css, angular, bootstrap5"
1810439,2022-11-21 12:43:43.470,"python, discord"
1810440,2022-11-16 15:58:53.977,"c, modem, glib"


In [4]:
preGPT['tags'] = preGPT['tags'].str.replace(',', ' ')
preGPT['tags']

0           nodeDOTjs  reactjs  express  axios  refreshtoken
1                                                   r  shiny
2          java  installation  compilation  project  oper...
3                       firebase  nuxtDOTjs  firebasehosting
4                                            r  dataDOTtable
                                 ...                        
1810437                           python  class  courseraapi
1810438                             css  angular  bootstrap5
1810439                                      python  discord
1810440                                       c  modem  glib
1810441                                regex  xml  notepad++
Name: tags, Length: 1810442, dtype: object

In [5]:
preGPT['tags'] = preGPT['tags'].str.split() 
preGPT['tags']

0          [nodeDOTjs, reactjs, express, axios, refreshto...
1                                                 [r, shiny]
2          [java, installation, compilation, project, ope...
3                     [firebase, nuxtDOTjs, firebasehosting]
4                                          [r, dataDOTtable]
                                 ...                        
1810437                         [python, class, courseraapi]
1810438                           [css, angular, bootstrap5]
1810439                                    [python, discord]
1810440                                     [c, modem, glib]
1810441                              [regex, xml, notepad++]
Name: tags, Length: 1810442, dtype: object

### 2) Create Co-Occurance Matrix

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from itertools import combinations

all_tags = [tag for sublist in preGPT['tags'] for tag in sublist]
all_tags

['nodeDOTjs',
 'reactjs',
 'express',
 'axios',
 'refreshtoken',
 'r',
 'shiny',
 'java',
 'installation',
 'compilation',
 'project',
 'operatorkeyword',
 'firebase',
 'nuxtDOTjs',
 'firebasehosting',
 'r',
 'dataDOTtable',
 'swift',
 'python',
 'discordDOTpy',
 'texttospeech',
 'reactjs',
 'frontend',
 'reactsimplemaps',
 'wordpress',
 'wordpresstheming',
 'elementor',
 'javascript',
 'axios',
 'googleplacesapi',
 'vbDOTnet',
 'r',
 'matrix',
 'plot',
 'python',
 'pandas',
 'go',
 'visualstudiocode',
 'python',
 'interface',
 'gettersetter',
 'typehinting',
 'pylance',
 'terraform',
 'gpgsignature',
 'hashicorp',
 'goreleaser',
 'python',
 'vlookup',
 'docker',
 'artifactory',
 'jfrogcontainerregistry',
 'r',
 'github',
 'shiny',
 'githubactions',
 'python',
 'django',
 'djangorestframework',
 'android',
 'androidactivity',
 'launchmode',
 'python',
 'python3DOTx',
 'reactnative',
 'expo',
 'sql',
 'postgresql',
 'commontableexpression',
 'aspDOTnetcore',
 'servicestack',
 'googleclo

In [7]:
unique_tags = list(set(all_tags))
tag_to_idx = {tag: idx for idx, tag in enumerate(unique_tags)}
tag_to_idx

{'ifmodifiedsince': 0,
 'github3DOTpy': 1,
 'spotinstances': 2,
 'pylatex': 3,
 'googlesigninaccount': 4,
 'apachenifiregistry': 5,
 'videoembedding': 6,
 'DOTnet6DOT0': 7,
 'objectrecognition': 8,
 'opentypesvgfont': 9,
 'azureiotdps': 10,
 'cabalnew': 11,
 'amazoncertificatemanager': 12,
 'gnucoreutils': 13,
 'clsx': 14,
 'acid': 15,
 'domrepeat': 16,
 'solanacli': 17,
 'addressablegem': 18,
 'datalayers': 19,
 'sqlreportingservice': 20,
 'tappayment': 21,
 'tensorflowhub': 22,
 'flutteranimatedlist': 23,
 'dumpdata': 24,
 'pydot': 25,
 'cda': 26,
 'msjetace': 27,
 'atlcom': 28,
 'processcontrol': 29,
 'cookiecutter': 30,
 'erdpy': 31,
 'mixedcontent': 32,
 'insertion': 33,
 'marshalling': 34,
 'firefoxprofile': 35,
 'javafx18': 36,
 'bism': 37,
 'acra': 38,
 'filewriter': 39,
 'gdbinit': 40,
 'androidkeystore': 41,
 'zindex': 42,
 'djangorestframeworkjwt': 43,
 'pullqueue': 44,
 'inequalities': 45,
 'ibinsync': 46,
 'digitaloceanapps': 47,
 'reactfire': 48,
 'archivetar': 49,
 'mode

In [8]:
# Make empty co-occurance matrix
co_occurance = np.zeros((len(unique_tags), len(unique_tags)))
co_occurance

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
co_occurance.shape

(44266, 44266)

In [10]:
for tags in preGPT['tags']:
    for tag1, tag2 in combinations(tags, 2):
        idx1, idx2 = tag_to_idx[tag1], tag_to_idx[tag2]
        co_occurance[idx1, idx2] += 1
        co_occurance[idx2, idx1] += 1
co_occurance

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [91]:
# save pickle (co-occurnace matrix)
#with open(file = 'co_occurance.pickle', mode = 'wb') as file:
#    pickle.dump(co_occurance, file)

In [54]:
with open(file = 'co_occurance.pickle', mode = 'rb') as file:
    co_occurance = pickle.load(file)

In [55]:
co_occurance

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
zero_sum_rows = np.where(np.sum(co_occurance, axis = 1 ) == 0)[0] # 207개

In [24]:
specific_rows = [row for i, row in enumerate(tag_to_idx) if i in zero_sum_rows]

In [25]:
specific_rows

['domoticz',
 'appofflineDOThtm',
 'mephisto',
 'pycxsimulator',
 'trackjs',
 'serverDOTtransfer',
 'cronofy',
 'gf',
 'googlecloudtalentsolution',
 'anko',
 'jung',
 'inform7',
 'magicchunks',
 'bitmapfonts',
 'libev',
 'togaf',
 'fbdev',
 'quireapi',
 'nvram',
 'pyexcelerator',
 'javaservicewrapper',
 'accurev',
 'backbase',
 'ydb',
 'fugue',
 'syndesis',
 'outlook2013',
 'freefem++',
 'voldemort',
 'origensdk',
 'microeditor',
 'sapsmp',
 'cofoundry',
 'collabnet',
 'sfx',
 'munin',
 'daru',
 'sablecc',
 'xshell',
 'prelink',
 'backupexec',
 'blitz3d',
 'webofthings',
 'easyrepro',
 'paylocity',
 'rythm',
 'im4java',
 'yii2dynagrid',
 'ton',
 'sqf',
 'ora00923',
 'eslintconfigvue',
 'saleaelogic',
 'ddms',
 'coreservices',
 'smartclient',
 'hyperon',
 'specs2',
 'ibmdatacap',
 'balsamiq',
 'ironworker',
 'dreamfactory',
 'qcubed',
 'yagna',
 'aquafold',
 'linkedinplugins',
 'cognitoforms',
 'darcs',
 'polarization',
 'alchemycms',
 'instantclickDOTio',
 'expressionencodersdk',
 'vis

In [ ]:
# 1) erase values with the row_sum == 0.
# 2) should also erase values in the list of index (tag name)
# 3) Normalize
# 4) Clustering.

### 3) Normalization

In [12]:
np.any(np.sum(co_occurance, axis = 1 ) == 0)

True

In [30]:
# Doesn't work (divdied by zero encountered)
# co_occurance = co_occurance / co_occurance.sum(axis = 1, keepdims = True)
#co_occurance

/tmp/ipykernel_4070951/1423437803.py:1: RuntimeWarning: divide by zero encountered in divide
  co_occurance = co_occurance / co_occurance.sum(axis = 1, keepdims = True)
/tmp/ipykernel_4070951/1423437803.py:1: RuntimeWarning: invalid value encountered in divide
  co_occurance = co_occurance / co_occurance.sum(axis = 1, keepdims = True)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [76]:
np.fill_diagonal(co_occurance, 1)
co_occurance

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

### 4) KMeans Clustering

In [77]:
from sklearn.cluster import KMeans
num_clusters = 50
kmeans = KMeans(n_clusters= num_clusters, random_state = 42)
clusters = kmeans.fit_predict(co_occurance)

tag_clusters = {tag: clusters[idx] for tag, idx in tag_to_idx.items()}
tag_clusters

/home/cglee/anaconda3/envs/ahnlab/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{'milo': 0,
 'odoowebsite': 0,
 'androidtextinputedittext': 0,
 'projectsandsolutions': 0,
 'threshold': 0,
 'fillable': 0,
 'googlereviews': 0,
 'numpyro': 0,
 'advancedrestclient': 0,
 'gracefuldegradation': 0,
 'dynamicsmarketing': 0,
 'glassfish6': 0,
 'therubyracer': 0,
 'djangolistview': 0,
 'amazoncloudwatchlogs': 0,
 'apachehop': 0,
 'automationtesting': 0,
 'cryengine': 0,
 'reduxmockstore': 0,
 'adjustandroid': 0,
 'exchangewebservices': 0,
 'visualstudioaddins': 0,
 'fakes3': 0,
 'djangoconstraints': 0,
 'wildfly22': 0,
 'jvmcodecache': 0,
 'accountmanagement': 0,
 'confirmationemail': 0,
 'shaderlab': 0,
 'openshift3': 0,
 'oracle11gr2': 0,
 'modproxyhtml': 0,
 'roberta': 0,
 'wikitude': 0,
 'jdal': 0,
 'iforels': 0,
 'rdrand': 0,
 'snowpark': 0,
 'qcustomplot': 0,
 'atcommand': 0,
 'mavennarplugin': 0,
 'npmlogin': 0,
 'azuredevtestlabs': 0,
 'yguard': 0,
 'dashshell': 0,
 'polynomialmath': 0,
 'googlemapsreact': 0,
 'dockercopy': 0,
 'sxssf': 0,
 'pymtp': 0,
 'srtm': 0,
 